# **1. Persiapan Environment**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Ekstraksi Data**

In [ ]:
import zipfile

# Extract the archive
local_zip = '/content/drive/MyDrive/Data Product 75.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp')
zip_ref.close()

# **3. Pemisahan Dataset**

In [ ]:
import os
import random
import shutil

# Path to the folder containing the images
folder_path = "/content/tmp/Data Product"

train_path = "/content/tmp/train_folder/"
val_path = "/content/tmp/val_folder/"
test_path = "/content/tmp/test_folder/"

# List of classes (subdirectories)
classes = os.listdir(folder_path)

# Create directories for train, val, and test sets
for class_name in classes:
    os.makedirs(os.path.join(train_path, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_path, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_path, class_name), exist_ok=True)

# Split the data into train, val, and test sets
for class_name in classes:
    class_path = os.path.join(folder_path, class_name)
    images = os.listdir(class_path)
    random.shuffle(images)

    train_size = int(0.70 * len(images))
    val_size = int(0.20 * len(images))

    train_images = images[:train_size]
    val_images = images[train_size:train_size + val_size]
    test_images = images[train_size + val_size:]

    # Copy images to train set
    for image in train_images:
        src = os.path.join(class_path, image)
        dst = os.path.join(train_path, class_name, image)
        shutil.copy(src, dst)

    # Copy images to val set
    for image in val_images:
        src = os.path.join(class_path, image)
        dst = os.path.join(val_path, class_name, image)
        shutil.copy(src, dst)

    # Copy images to test set
    for image in test_images:
        src = os.path.join(class_path, image)
        dst = os.path.join(test_path, class_name, image)
        shutil.copy(src, dst)

# **4. Augmentasi Data dan Persiapan Generator**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Ukuran gambar untuk model
img_width, img_height = 150, 150
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel [0, 1]
    rotation_range=20,       # rotate gambar
    width_shift_range=0.2,   # shift horizontally
    height_shift_range=0.2,  # shift vertically
    horizontal_flip=True,    # flip horizontally
    shear_range=0.2,         # shear gambar
    zoom_range=0.2,           # zoom gambar
    brightness_range=[0.8, 1.2]
)

# rescale untuk validation/test
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    val_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 2660 images belonging to 76 classes.
Found 760 images belonging to 76 classes.
Found 380 images belonging to 76 classes.


# **5. Pembuatan Model**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

# **6. Pelatihan Model**

In [ ]:
from keras.callbacks import EarlyStopping

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/25
83/83 [==============================] - 57s 612ms/step - loss: 4.3322 - accuracy: 0.0213 - val_loss: 4.2313 - val_accuracy: 0.0204
Epoch 2/25
83/83 [==============================] - 52s 633ms/step - loss: 3.9322 - accuracy: 0.0772 - val_loss: 2.9334 - val_accuracy: 0.2459
Epoch 3/25
83/83 [==============================] - 50s 603ms/step - loss: 3.1702 - accuracy: 0.1838 - val_loss: 1.6478 - val_accuracy: 0.5965
Epoch 4/25
83/83 [==============================] - 47s 568ms/step - loss: 2.4280 - accuracy: 0.3128 - val_loss: 0.9862 - val_accuracy: 0.7500
Epoch 5/25
83/83 [==============================] - 49s 592ms/step - loss: 1.9389 - accuracy: 0.4209 - val_loss: 0.5465 - val_accuracy: 0.8546
Epoch 6/25
83/83 [==============================] - 51s 617ms/step - loss: 1.5744 - accuracy: 0.5175 - val_loss: 0.3365 - val_accuracy: 0.9212
Epoch 7/25
83/83 [==============================] - 50s 601ms/step - loss: 1.3285 - accuracy: 0.5875 - val_loss: 0.2738 - val_accuracy: 0.9226

# **7. Evaluasi dan Penyimpanan Model**

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

12/12 [==============================] - 4s 294ms/step - loss: 0.0233 - accuracy: 0.9974
Test Loss: 0.023268526419997215
Test Accuracy: 0.9973683953285217


In [ ]:
model.save("/content/drive/MyDrive/model klasifikasi minuman/model_v1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import json

class_indices = train_generator.class_indices  # Ambil pemetaan kelas
class_indices_json = json.dumps(class_indices, indent=4)  # Ubah ke JSON dengan indentasi

# Simpan ke file
with open("class_indices.json", "w") as f:
    f.write(class_indices_json)

# **8. Prediksi**


In [ ]:
class_indices = validation_generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
from IPython.display import Image, display

uploaded = files.upload()

for fn in uploaded.keys():

    # Display the uploaded image
    display(Image(fn))

    # Prediction
    path = fn
    img = load_img(path, target_size=(150, 150))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    images = np.vstack([x])
    classes = model.predict(images)
    predicted_class_index = np.argmax(classes, axis=1)[0]
    predicted_class_label = index_to_class[predicted_class_index]
    print(fn)
    print(predicted_class_label)